In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium, fiona, branca

In [2]:
# Crear un geodataframe que tenga como geometry los poligonos de las parroquias de la ciudad de Guayaquil. 
gdf1 = gpd.read_file("../input/zonas_inec.geojson")
gdf1 = gdf1.rename(columns={'DPA_NOMBRE': 'Name'})

# Además, debe tenen una columna Name con el nombre de la parroquia, y debe tener una columna preciomt2 con el
# precio por metro cuadrado promedio en el que se venden las casas en cada parroquia.
df = pd.read_csv("../input/ec_properties.csv")
df.drop_duplicates(subset=["surface_total","lat","lon"], keep="last", inplace = True)
df.dropna(subset=["surface_total","price","lat","lon"],inplace=True)
df = df.loc[df["surface_total"]>0,:]
df["preciomt2"] = df["price"] / df["surface_total"]
gcasas = gpd.GeoDataFrame(df,geometry= gpd.points_from_xy(df["lon"],df["lat"]))
gcasas.crs = gdf1.crs
gjoin = gpd.sjoin(gcasas,gdf1)
grouped = gjoin.groupby('Name').agg({
    'preciomt2': 'mean',    
    'geometry': lambda x: x.unary_union.centroid  
}).reset_index()

# Asegúrese de eliminar outliers previo a realizar este análisis
# Su geodataframe guardelo en la variable gdf y ejecute las siguiente celdas:
gdf = gpd.GeoDataFrame(grouped, geometry='geometry')

In [7]:
gdf

,Name,preciomt2,geometry
0,AMBATO,772.031219,POINT (-78.61781 -1.25057)
1,ASCAZUBI,793.650794,POINT (-78.29290 -0.07852)
2,ATACAMES,40.000000,POINT (-79.85328 0.87104)
3,BABA,255.434783,POINT (-79.68027 -1.78260)
4,BABAHOYO,1370.227978,POINT (-79.53465 -1.80193)
...,...,...,...
72,URBANIZACION LOS SAUCES,725.352113,POINT (-78.44774 -0.02899)
73,VILCABAMBA,14.900000,POINT (-79.22538 -4.26233)
74,VILLAS DEL MAR,228.000000,POINT (-80.92395 -2.26257)
75,YARUQUI,23.854847,POINT (-78.31576 -0.16303)


In [4]:
#crea el mapa con un tile service
# centroid=gdf.geometry.centroid
# mapbox_api_key = r"pk.eyJ1Ijoic3RhbWVuIiwiYSI6IlpkZEtuS1EifQ.jiH_c9ShtBwtqH9RdG40mw"; tileset_ID_str = "streets-v11"; tilesize_pixels = "256";
# m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=12,
#              tiles = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_api_key}",
#              attr='MAPBOX')

In [5]:
# field = "preciomt2"
# label = "precio por metro cuadrado"
# colormap = branca.colormap.linear.YlOrBr_04.to_step(4).scale(gdf[field].min(),gdf[field].max())
# colormap.caption = label
# layer = folium.GeoJson('output.geojson',
#         name=label,
#         style_function=lambda x: {"weight":2, 'color':'black','fillColor':colormap(x['properties'][field]), 'fillOpacity':1},
#         highlight_function=lambda x: {'weight':5, 'color':'black'},
#         smooth_factor=2.0,
#         tooltip=folium.features.GeoJsonTooltip(fields=['Name',field],
#                                       aliases=['Name',label], 
#                                       labels=True, 
#                                       sticky=True,
#                                       localize=True
#                                      ),
#         show = True,
#         overlay = True
#         )
# m.add_child(layer)
# m.add_child(colormap)

In [6]:
# m.save('choropleth.html')